In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

from PIL import Image
from keras.layers import Dense, Flatten, Reshape, Conv2DTranspose, Conv2D, BatchNormalization, LeakyReLU, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
resized_dim = (112, 168)
BUFFER_SIZE = 60000
BATCH_SIZE = 32
image_directory = 'data/processed_images_v2'

Input Data recuperation

In [ ]:
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory)]

image_list = []
for image_path in image_paths:
    with Image.open(image_path) as img:
        pixel_values = np.array(img.resize(resized_dim))

    image_list.append(pixel_values)

image_np_array = np.array(image_list)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(image_np_array).shuffle(BUFFER_SIZE)

Print first image of the dataset

In [ ]:
iterator = iter(train_dataset.take(1))

first_image = next(iterator)

plt.imshow(first_image)
plt.axis('off')
plt.show()
    

In [ ]:
first_image.shape

In [ ]:
train_dataset = train_dataset.batch(BATCH_SIZE)

Parameters

In [ ]:
image_shape = (168, 112, 3)
z_dim = 100

Generator Model

In [ ]:
def build_generator(z_dim):
    model = tf.keras.Sequential()

    # Commencez par une couche Dense qui prend l'input de la dimension de l'espace latent z
    # La première couche Dense est un pré-traitement pour obtenir la forme désirée
    model.add(Dense(21*14*256, use_bias=False, input_shape=(z_dim,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Redimensionnement pour passer à la forme 21x14x256
    model.add(Reshape((21, 14, 256)))

    # Upsampling à 42x28
    model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 42, 28, 128)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Upsampling à 84x56
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 84, 56, 64)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Dernier upsampling pour obtenir 168x112
    model.add(Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 168, 112, 3)

    return model

In [ ]:
generator = build_generator(z_dim)

noise = tf.random.normal([1, z_dim])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0])

Discriminator Model

In [ ]:
def build_discriminator(image_shape):
    model = tf.keras.Sequential()
    
    # Couche d'entrée
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=image_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    # Couche intermédiaire
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    # Couche de sortie
    model.add(Flatten())
    model.add(Dense(1))

    return model

In [ ]:
discriminator = build_discriminator(image_shape)
decision = discriminator(generated_image)
print (decision)

In [ ]:
# Définissez la dimension de l'espace latent z
z_dim = 100

# Créez le générateur et le discriminateur avec les nouvelles formes
generator = build_generator(z_dim)
discriminator = build_discriminator((168, 112, 3))

# Compilez le discriminateur
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# Créez le modèle GAN complet
discriminator.trainable = False  # Très important pour s'assurer que le discriminateur n'est pas entraîné lors de l'entraînement du générateur via le modèle GAN
gan_input = tf.keras.Input(shape=(z_dim,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)

# Compilez le modèle GAN complet
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
def display_sample_images(generator, z_dim, epoch):
    noise = tf.random.normal([1, z_dim])
    generated_images = generator(noise, training=False)
    generated_images = (generated_images + 1) / 2 

    plt.imshow(generated_images)
    plt.axis('off')
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()

In [ ]:
def train_gan(generator, discriminator, gan, dataset, z_dim, epochs=100, batch_size=32, save_interval=5):
    size_of_dataset = sum(1 for _ in dataset)
    steps_per_epoch = size_of_dataset // batch_size

    for epoch in range(1, epochs + 1):
        for _, real_images in enumerate(dataset.take(steps_per_epoch)):
            noise = tf.random.normal([batch_size, z_dim])

            with tf.GradientTape() as disc_tape:
                fake_images = generator(noise, training=False)
                real_output = discriminator(real_images, training=True)
                fake_output = discriminator(fake_images, training=True)

                disc_loss = discriminator_loss(real_output, fake_output)

            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

            noise = tf.random.normal([batch_size, z_dim])

            with tf.GradientTape() as gen_tape:
                fake_images = generator(noise, training=True)
                fake_output = discriminator(fake_images, training=False)
                gen_loss = generator_loss(fake_output)

            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        print(f'Époque {epoch}/{epochs}, Discriminator Loss: {disc_loss.numpy()}, Generator Loss: {gen_loss.numpy()}')

        if epoch % save_interval == 0:
            display_sample_images(generator, z_dim, epoch)

In [ ]:
train_gan(generator, discriminator, gan, train_dataset, z_dim, epochs=10000, batch_size=BATCH_SIZE)

In [ ]:
noise = tf.random.normal([1, z_dim])

generated_image = generator(noise, training=False)

generated_image = (generated_image.numpy().squeeze() * 127.5 + 127.5).astype(np.uint8)

plt.imshow(generated_image)
plt.axis('off')
plt.show()